In [ ]:
#importing the necessary libraries
import requests, csv, pandas as pd, pprint, time
from bs4 import BeautifulSoup
import lxml,html5lib
import re 
import time
import numpy as np

In [ ]:
#Script to get all ps4 title names from metacritic website

def titles():
    titles_list = []
    for i in range(0,28):
        url = 'https://www.metacritic.com/browse/games/release-date/available/ps4/metascore?page='+str(i)
        #print(url)
        userAgent = {'User-agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=userAgent)
        req=requests.get(url,headers=userAgent)
        content=req.text
        print("Request " + str(i) + " made")
        
        time.sleep(5) # To avoid blocking from website causing remote server to overload
        games_list= []
        soup=BeautifulSoup(content)
        games_list = list(soup.find_all('a', {"class":"title"}))
        for i in range(len(games_list)):
            titles_list.append(str(games_list[i]).split('<h3>')[1].split('</h3>')[0])
    #print(titles_list[-1])
    return titles_list

After getting the title names of all games we iterate through all the games and fetch the user reviews

In [1]:
#Manual checking - this function was run manually 2000+ times due to easily debug issues.
def scrape_reviews():
    title_name = []
    users_list = []
    ratings_list = []
    errors = []
    for i in range(0,10):
        url='https://www.metacritic.com/game/playstation-4/tony-hawks-pro-skater-5/user-reviews?page='+str(i)
        userAgent = {'User-agent': 'Mozilla/5.0'}
        req=requests.get(url,headers=userAgent)
        content=req.text
        print("Request " + str(i) + " made")
        #print("request made")
        time.sleep(5)
        soup=BeautifulSoup(content)
        title=list(soup.find_all('h1'))

    
        users = list(soup.find_all('a',text=re.compile(r'user')))
        ratings = list(soup.find_all('div', {"class":re.compile(r'metascore_w user medium game')}))
        if len(users) != len(ratings): # This condition is to check if data has been scraped properly at each iteration
            errors.append(i)
    for i in range(len(ratings)):
        users_list.append(str(users[i]).split('"')[1].split('"')[0])
        ratings_list.append(int(str(ratings[i]).split(">")[1].split('<')[0]))
        title_name.append(str(title[0]).split('<h1>')[1].split('</h1>')[0])
    print(len(users))
    print(len(ratings))
    return title_name, users_list, ratings_list

In [ ]:
# This block of code appends new game reviews to the exisitng excel 
df = pd.read_excel('C:/Users/hdeok/Desktop/game_data/Game_ratings_PS4.xlsx',index_col=0)

df1 = pd.DataFrame(users_list, columns=['Users'])
df1['Game_name'] = title_name
df1['user_rating'] = ratings_list 

df = df.append(df1)

df = df.reset_index()
df.drop(['index'],axis=1,inplace=True)
df.to_excel('C:/Users/hdeok/Desktop/game_data/rating.xlsx') 